In [ ]:
# Colab setup code block
try:
    import google.colab
    print("Running on Google Colab. Setting up environment...")
    
    import os
    if not os.path.exists('CMAI-E91-Students'):
        !git clone https://github.com/algolab-quantique/CMAI-E91-Students.git

    os.chdir('CMAI-E91-Students/Part_2_E91')
    !pip install -r ../requirements.txt -q
    print("Environment setup complete!")
except ImportError:
    print("Not running on Google Colab. Skipping setup.")


# Protocole de distribution de clé quantique E91 – Atelier pratique

**Objectifs d'apprentissage :**
- Implémenter le protocole E91 complet
- Utiliser l'inégalité de Bell CHSH pour détecter l'espionnage
- Générer une clé partagée sécurisée
- Chiffrer et déchiffrer des messages

---

## Rappel rapide (des diapositives)

**Étapes du protocole E91 :**
1. Générer des paires de Bell intriquées
2. Alice et Bob choisissent au hasard leurs bases de mesure
3. Mesurer toutes les paires
4. Trier les résultats : mêmes bases → clé, bases CHSH → test de sécurité
5. Lancer le test CHSH : si $|S| > 2$ → sécurisé !
6. Extraire la clé à partir des mesures dans les bases concordantes

---


---
## Points clés du protocole E91 (Rappel)

- **Bases :** Alice $\{\color{red}{0^\circ}$, $45^\circ$, $\color{red}{90^\circ}\}$, Bob $\{\color{red}{45^\circ}$, $90^\circ$, $\color{red}{135^\circ}\}$
- **Paires pour la clé :** $(45^\circ, 45^\circ)$ et $(90^\circ, 90^\circ)$
- **Paires pour le test CHSH :** $(0^\circ, 90^\circ)$, $(0^\circ, 135^\circ)$, $(45^\circ, 90^\circ)$, $(45^\circ, 135^\circ)$
- **Rejeter :** toutes les autres combinaisons de bases
- **Sécurité :** calculer $S$ à partir des paires CHSH ; sécurisé si $|S| > 2$
- **État singulet :** les résultats sont anti-corrélés, donc Bob inverse les bits pour la clé
---


## Configuration

**Rappel :** La plupart des fonctions utilitaires viennent du CHSH — elles sont fournies comme un module prêt à l'emploi. L'**étape de tamisage (sifting)** est le cœur du E91, c'est là-dessus que nous allons nous concentrer !


In [1]:
import random
import numpy as np
from qiskit import QuantumCircuit
import sys
sys.path.append('utils')
import encryption_algorithms as enc

# Import solved CHSH functions from the utils module
from utils.chsh_core import *

print(" Setup complete!")
print("\n Imported CHSH functions from utils/chsh_core.py:")
print("   • run_circuit")
print("   • create_bell_pair_singlet_state") 
print("   • apply_basis_transformation")
print("   • measure_bell_pair")
print("   • create_eavesdropped_state")
print("   • organize_measurements_by_basis")
print("   • calculate_correlations")
print("   • calculate_chsh_value")

# Set seed for reproducibility  
GLOBAL_SEED = 91
random.seed(GLOBAL_SEED)
np.random.seed(GLOBAL_SEED)

# E91 Protocol Constants
ALICE_BASES = ['0', '45', '90']      # Alice's available bases
BOB_BASES = ['45', '90', '135']      # Bob's available bases
ALICE_CHSH_BASES = ['0', '90']
BOB_CHSH_BASES = ['45', '135']
CHSH_BASIS_PAIRS = [(a, b) for a in ALICE_CHSH_BASES for b in BOB_CHSH_BASES]

# Interpretation:
#   |S| < 2.0       → Classical regime (no violation)
#   2.0 < |S| < 2.5 → Caution: theoretical violation, but may be due to noise or
#                     insufficient statistics (this boundary is a pedagogical choice,
#                     not a scientifically established threshold)
#   |S| > 2.5       → Robust violation, clearly quantum
BELL_INEQUALITY_THRESHOLD = 2.5

EVE_PERCENTAGE_COMPROMISED = 0.7 # Eve intercepts 70% of the pairs

print(f"\nAlice's bases: {ALICE_BASES}")
print(f"Bob's bases: {BOB_BASES}")
print(f"Key generation pairs: (45,45) and (90,90)")
print(f"CHSH test basis pairs : {CHSH_BASIS_PAIRS}")

 Setup complete!

 Imported CHSH functions from utils/chsh_core.py:
   • run_circuit
   • create_bell_pair_singlet_state
   • apply_basis_transformation
   • measure_bell_pair
   • create_eavesdropped_state
   • organize_measurements_by_basis
   • calculate_correlations
   • calculate_chsh_value

Alice's bases: ['0', '45', '90']
Bob's bases: ['45', '90', '135']
Key generation pairs: (45,45) and (90,90)
CHSH test basis pairs : [('0', '45'), ('0', '135'), ('90', '45'), ('90', '135')]


## 1) Générer des bases aléatoires

Alice et Bob choisissent chacun, pour chaque qubit, une base de mesure de manière aléatoire.  
Pourquoi : des choix indépendants et aléatoires empêchent Eve de prédire les paramètres de mesure.


In [2]:
def generate_random_bases(length: int, options: list) -> list:
    """
    Generate a list of random bases.
    
    Args:
        length: Number of bases to generate
        options: List of possible bases (e.g., ['0', '45', '90'])
    
    Returns:
        List of randomly chosen bases
    """
    # Simply choose 'length' random bases from the options
    return [random.choice(options) for _ in range(length)]

In [ ]:
# Test
num_random_basis = 10
test_bases = generate_random_bases(num_random_basis, ['0', '45', '90'])
print(f"Generated bases: {test_bases}")
print(f"Expected: {num_random_basis} random values from ['0', '45', '90']")

Generated bases: ['0', '90', '0', '90', '90', '0', '45', '45', '90', '45']
Expected: 10 random values from ['0', '45', '90']


## 2) Créer des paires de Bell

Pourquoi : le protocole E91 commence par générer de nombreuses paires intriquées identiques (même principe que pour le CHSH).


In [4]:
def create_list_bell_pairs(num_pairs: int) -> list:
    """
    Create a list of Bell pair circuits.
    
    Args:
        num_pairs: Number of Bell pairs to create
    
    Returns:
        List of QuantumCircuit objects (Bell pairs)
    """
    # Create 'num_pairs' identical Bell states using the function from NB1
    return [create_bell_pair_singlet_state() for _ in range(num_pairs)]



## 3) Mesurer toutes les paires

In [5]:
def measure_all_pairs(bell_pairs: list, alice_bases: list, bob_bases: list) -> list:
    """
    Measure each Bell pair with the corresponding bases.
    
    Returns:
        List of measurement results ('00', '01', '10', '11')
    """
    results = []
    
    # Loop through every pair and measure it using the chosen bases
    for qc, a_base, b_base in zip(bell_pairs, alice_bases, bob_bases):
        # measure_bell_pair is the function from Notebook 1
        result = measure_bell_pair(qc, a_base, b_base)
        results.append(result)
    
    return results


## 4) Trier les résultats (Génération de clé vs Test de Bell)

**Règles de tri :**
- Mêmes bases (45,45) ou (90,90) → **Génération de clé**
- Bases CHSH → **Test de Bell**
- Autres combinaisons → **Rejet**

**C'est le cœur du protocole E91** — cela détermine quelles données deviennent la clé et lesquelles servent au test de sécurité.

Exercice : implémentez `extract_e91_key_and_bell_test_data()`


### Intuition du tri (pourquoi cela fonctionne)

- **Mêmes bases** → forte (anti-)corrélation → sûr pour les bits de la clé
- **Bases différentes** → plus aléatoires → utile pour le test de Bell ou à rejeter

Cela reflète ce qui se passe avec les polariseurs dans l’expérience en laboratoire.


In [ ]:
def extract_e91_key_and_bell_test_data(results: list, alice_bases: list, bob_bases: list) -> dict:
    """
    Sift measurement results into key generation and Bell test data.
    
    Returns:
        dict with:
            'key_results': results for key generation
            'chsh_results': results for Bell test
            'chsh_alice_bases': Alice's bases for Bell test
            'chsh_bob_bases': Bob's bases for Bell test
    """
    key_results = [] # keep output results for same bases
    chsh_results = [] # keep output results for bases combinations that belong to CHSH test
    chsh_alice_bases = [] # keep alice bases that are in (combinations that belong to CHSH test)
    chsh_bob_bases = [] # same as alice
    # note : at index i, we have chsh output related to alice and bob base
    
    for i, (result, a_base, b_base) in enumerate(zip(results, alice_bases, bob_bases)):
        
        # TODO: Check if same basis (a_base == b_base)
        # If yes, add to key_results
        
        # TODO: Check if (a_base, b_base) is in CHSH_BASIS_PAIRS
        # If yes, add to chsh_results, chsh_alice_bases, chsh_bob_bases
        
        # Otherwise: discard
        pass
    
    return {
        'key_results': key_results,
        'chsh_results': chsh_results,
        'chsh_alice_bases': chsh_alice_bases,
        'chsh_bob_bases': chsh_bob_bases,
    }

## 5) Vérifier l'espionnage

Utilisez les données du test de Bell pour vérifier la sécurité.  
Si $|S| > 2$, l'intrication est intacte et le canal est sécurisé.

*(Cette cellule réutilise la logique CHSH du notebook 1. Fournie pour gagner du temps.)*


In [ ]:
def check_for_eavesdropping(chsh_results: list, chsh_alice_bases: list, chsh_bob_bases: list) -> dict:
    """
    Run CHSH test to check for eavesdropping.
    """
    # 1. Organize the raw measurement results by basis pair
    bell_results = organize_measurements_by_basis(chsh_results, chsh_alice_bases, chsh_bob_bases)
    
    # 2. Calculate the correlation for each basis pair
    correlations = calculate_correlations(bell_results)
    
    # 3. Compute the CHSH 'S' value
    chsh_value = calculate_chsh_value(correlations)
    
    # 4. Check against the threshold (2.5)
    is_secure = chsh_value > BELL_INEQUALITY_THRESHOLD
    
    return {
        'chsh_value': chsh_value,
        'is_secure': is_secure
    }


## 6) Protocole E91 complet

Mettons tout cela ensemble !  
Cette orchestration montre l’ensemble du processus, de l’intrication jusqu’à l’obtention d’une clé partagée.

Exercice : complétez `run_e91_protocol()`


In [ ]:
EVE_PERCENTAGE_COMPROMISED = 0.7

def run_e91_protocol(num_pairs: int = 2000, eavesdropping: bool = False) -> str:
    """
    Run the complete E91 QKD protocol.
    
    Args:
        num_pairs: Number of Bell pairs to generate
        eavesdropping: If True, simulate Eve intercepting pairs
    
    Returns:
        Shared secret key (string of 0s and 1s) or None if insecure
    """
    print("="*60)
    print("E91 QUANTUM KEY DISTRIBUTION PROTOCOL")
    print("="*60)
    
    # Step 1: Generate Bell pairs
    print(f"\n1. Generating {num_pairs} Bell pairs...")
    bell_pairs = create_list_bell_pairs(num_pairs)
    
    # Step 2: Simulate eavesdropping (optional)
    if eavesdropping:
        compromised = int(num_pairs * EVE_PERCENTAGE_COMPROMISED)
        print(f"\n EVE intercepts {compromised} pairs ({EVE_PERCENTAGE_COMPROMISED*100}%)")
        bell_pairs = [create_eavesdropped_state(qc) for qc in bell_pairs[:compromised]] + bell_pairs[compromised:]
    
    # Step 3: Generate random bases
    print("\n2. Alice and Bob choose random bases...")
    alice_bases = generate_random_bases(num_pairs, ALICE_BASES)
    bob_bases = generate_random_bases(num_pairs, BOB_BASES)
    
    # Step 4: Measure all pairs
    print("\n3. Measuring all pairs...")
    results = measure_all_pairs(bell_pairs, alice_bases, bob_bases)
    
    # Step 5: Sift results
    print("\n4. Sifting results...")
    data = extract_e91_key_and_bell_test_data(results, alice_bases, bob_bases)
    
    print(f"   Key generation pairs: {len(data['key_results'])}")
    print(f"   Bell test pairs: {len(data['chsh_results'])}")
    
    # Step 6: Security check
    print("\n5. Running CHSH security test...")
    security = check_for_eavesdropping(
        data['chsh_results'],
        data['chsh_alice_bases'],
        data['chsh_bob_bases']
    )
    
    print(f"\n   CHSH Value: {security['chsh_value']:.4f}")
    print(f"   Classical limit: 2.0")
    
    if not security['is_secure']:
        print("\n SECURITY CHECK FAILED!")
        print("   Possible eavesdropping detected.")
        print("   Key generation ABORTED.")
        return None
    
    print("\n SECURITY CHECK PASSED!")
    
    # Step 7: Extract key
    print("\n6. Extracting shared key...")
    
    # Alice's bits (first bit of each result)
    alice_key = ''.join([r[0] for r in data['key_results']])
    
    # Bob's bits (second bit of each result)
    # For singlet state: anti-correlated, so Bob flips his bits
    bob_key = ''.join(['1' if r[1] == '0' else '0' for r in data['key_results']])
    
    # Verify keys match
    if alice_key != bob_key:
        mismatches = sum(a != b for a, b in zip(alice_key, bob_key))
        print(f"\n Warning: {mismatches} bit mismatches (noise or Eve)")
    else:
        print("\n Keys match perfectly!")
    
    print(f"\n   Key length: {len(alice_key)} bits")
    print(f"   Key (first 50 bits: {alice_key[:50]}...")
    
    return alice_key


## Lancer le protocole E91 (sans espionnage)

In [ ]:
# Run without Eve
key = run_e91_protocol(num_pairs=2000, eavesdropping=False)

## Lancer le protocole E91 (avec espionnage)

In [ ]:
# Reset seed
random.seed(GLOBAL_SEED)
np.random.seed(GLOBAL_SEED)
MANUAL_SIMULATOR_SEED_COUNTER = GLOBAL_SEED

# Run with Eve, we use 800 pairs, just to show the concept 
key_with_eve = run_e91_protocol(num_pairs=800, eavesdropping=True)

## Chiffrer et déchiffrer des messages

In [ ]:
# use the generated key to encrypt and decrypt a message using XOR
# I use the key generated without Eve for encryption, as the one with Eve is likely compromised
message = '"What I cannot create, I do not understand." - Richard Feynman'
print(f"\nOriginal: {message}")

encrypted = enc.encrypt_xor_repeating_key(message, key)
print(f"\nEncrypted (hex): {encrypted[:50]}...")

decrypted = enc.decrypt_xor_repeating_key(encrypted, key)
print(f"\nDecrypted: {decrypted}")

---
## Défi : Déchiffrer les messages secrets !

Le fichier `encrypted_messages.txt` contient des messages chiffrés à l’aide de la clé E91.  
**Votre mission : les déchiffrer !**


In [ ]:
def decrypt_messages_from_file(key: str, filename: str):
    """Decrypt all messages from a file."""
    print(f"\nDecrypting messages from {filename}...\n")
    
    try:
        with open(filename, 'r') as f:
            for line in f:
                if ': ' in line:
                    msg_id, encrypted = line.strip().split(': ', 1)
                    decrypted = enc.decrypt_xor_repeating_key(encrypted, key)
                    print(f"{msg_id}: {decrypted}")
    except FileNotFoundError:
        print(f" File not found: {filename}")


# Decrypt the messages
if key:
    decrypt_messages_from_file(key, 'encrypted_messages.txt')

---
## Résumé

**Ce que nous avons implémenté :**
1. Protocole complet de distribution de clé quantique E91
2. Sélection aléatoire des bases pour Alice et Bob
3. Tri des résultats (clé vs test de Bell)
4. Vérification de la sécurité CHSH
5. Extraction de la clé avec gestion de l’anti-corrélation
6. Chiffrement et déchiffrement de messages

**Aperçu clé :** le test de Bell CHSH détecte l’espionnage !
- Pas d’Eve → $|S| \approx 2,83$ → Sécurisé 
- Avec Eve → $|S|$ chute → Détecté ! 

---

**Ensuite :** complétez le devoir à la maison en utilisant un **état de Bell différent** !
